In [1]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab :(")

REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
! pip install -r requirements.txt

In [5]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from functools import partial

In [6]:
with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
  		dict_country_cdandlp = pickle.load(f)

In [ ]:
dict_country_cdandlp

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
"""# Inputs"""

baseline = pd.read_pickle(REP_INTERMED + 'baseline.sav').drop([	'released_date'], axis=1)

with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
  dict_words = pickle.load(f)

with open(REP_INTERMED + 'dict_country_cdandlp.pkl', 'rb') as f:
  		dict_country_cdandlp = pickle.load(f)
    
dict_country_cdandlp_france = dict_country_cdandlp['France']
df_sample= pd.DataFrame(dict_country_cdandlp_france, columns=['pays_clean', 'text', 'n_ref', 'artiste', 'titre'])

analyzer = 'char'
ngram_range = (2,2)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)


In [ ]:
def knn(start , 
        end, 
        baseline = baseline, 
        dict_words=dict_words,
        dict_country_cdandlp =dict_country_cdandlp_france
        ) :

  df_sample= pd.DataFrame(dict_country_cdandlp_france[start: end], columns=['pays_clean', 'text', 'n_ref', 'artiste', 'titre'])
  result = [ level1_knn_words(row_index, df_sample, baseline, dict_words)  for row_index in range(len(df_sample)) ]
  
  with open(REP_INTERMED + str(end) + '_France_words_knn.pkl', 'wb') as  f1:
    pickle.dump(result, f1) 

In [ ]:
taille = len(dict_country_cdandlp_france)
pas = int(taille/30)
list_tmp = [ indices for indices in range(0,  taille, pas) ]

start_time = datetime.now()
[ knn(start, end) for start , end in zip(list_tmp,list_tmp[1:])]
# LAST 
knn(21030, 21045)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 4:17:01.355972


In [ ]:
knn(21030, 21045)

In [ ]:
from pathlib import Path

In [ ]:
## Concaténation et lecture des csv
dfs = [pd.concat(pd.read_pickle(path), ignore_index=True) for path in list(Path('Intermed').glob('*France_words_knn*'))]
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_pickle(REP_INTERMED + 'results_France_concat.pkl')
df_final.to_excel("resultats_knn_france.xlsx", index=False)